# Токенизация и подсчет количества слов

### Сколько слов в этом предложении?

* На дворе трава, на траве дрова, не руби дрова на траве двора.*

<b> 12 токенов </b> На, дворе, трава, на, траве, дрова, не, руби, дрова, на, траве, двора<br>
<b> 8 - 9 типов </b> Н/на, дворе, трава, траве, дрова, не, руби, дрова<br>
<b> 6 лексем </b>на, не, двор, трава, дрова, рубить

#### Токен и тип
<b>Тип</b> - уникальное слово из текста<br>
<b>Токен</b> - тип и его позиция в тексте

#### Обозначения
N = число токенов
V - словарь (все типы)
|V| = число типов в словаре

<b>Как связаны N и |V|?</b>

<b>Закон Ципфа</b>
В любом достаточно большом тексте ранг типа обратно пропорционален его частотности $f = \frac {a}{r}$<br>
$f$ - частота типа, $r$ - ранг типа, $a$ - параметр для славянских языков - около 0.07

<b>Закон Хипса</b>
С увеличением длины текста {количества токнов}, количество типов увеличивается в соответствии с законом $|V| =  {K}*{N^b}$<br>
$N$ - число токенов, $|V|$ - количество типов в словаре, $K,b$ - параметры, обычно K $\in$ [10, 100], b $\in$ [0.4, 0.6] 

## Анализ сообщений VK.com

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import pandas as pd

df = pd.read_csv('datasets/nlp/vk_texts_with_sources.csv', usecols=['text', 'source'])

df.text.dropna(inplace=True)
df.head()

## Предварительный анализ коллекции

### Средняя длина текстов

In [ ]:
df.text = df.text.astype(str)
len_data = df.text.apply(len)
len_data.describe()

### Количество текстов разных пабликов


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

counts = df.source.value_counts()
values = counts.tolist()
labels = counts.index.tolist()

y_pos = np.arange(len(labels))

plt.bar(y_pos, values, align='center', alpha=.5)
plt.xticks(y_pos, range(len(labels)))

plt.show()

### Длины текстов в символах

In [ ]:
fig, ax = plt.subplots()

length = len_data[len_data < 10000].tolist()

n, bins, patches = ax.hist(length)

fig.show()

### Токенизация

Используем регулярные выражения, чтобы разбить текст на слова

In [ ]:
import re

regex = re.compile('[А-Яа-я]+')

def words_only(text, regex=regex):
    try:
        return ' '.join(regex.findall(text))
    except:
        return ''
    
df.text = df.text.str.lower()
df.text = df.text.apply(words_only)

df.text.iloc[0]

### Самые частые слова

In [ ]:
from nltk import FreqDist

n_types = []
n_tokens = []
fd = FreqDist()

for index, row in df.iterrows():
    tokens = row['text'].split()
    fd.update(tokens)
    n_types.append(len(fd))
    n_tokens.append(sum(fd.values()))

for i in fd.most_common(10):
    print(i)

### Закон Ципфа

In [ ]:
freqs = list(fd.values())
freqs = sorted(freqs, reverse=True)

fig, ax = plt.subplots()
ax.plot(freqs[:300], range(300))
plt.show()

### Сегментация предложений
"!","?" как правило однозначны, проблемы возникают с "."
Бинарный классификатор для сегментации преллодений для каждой точки "." лпределить, является ли она концом 
предложения или нет.

In [ ]:
from nltk.tokenize import sent_tokenize

text = "Первое предложение. Второе предложение! И, наконец, третье? В четвертом предложении г. Москва встречается т. к. это город."
sents = sent_tokenize(text)

print(len(sents))
print(*sents, sep='\n')

In [ ]:
import nltk

nltk.download('punkt')

### Задание 2

Посчитайте количество предложений, токенов и типов из файла task2.txt, Сохраните список токенов в массив tokens.

In [ ]:
import re
from nltk import FreqDist

fd = FreqDist()
regex = re.compile('[А-Яа-я]+')

def words_only(text, regex=regex):
    try:
        return ' '.join(regex.findall(text))
    except:
        return ''
    
text = ' '.join([line.strip() for line in open('datasets/nlp/task2.txt', encoding='utf8')])
tokens = words_only(text)
sents = sent_tokenize(text)
print(len(sents))
d1 = nltk.FreqDist(tokens)
print(d1)

### Частотный анализ текста

In [ ]:
import nltk

d1 = nltk.FreqDist(tokens) # частотный словарь для текстов
d1.most_common(10) # токен и  его количество появлений в тексте


## Морфологический анализ

### Задачи морфологи ческого аннализа

* Разбор слов - определение нормальной формы (леммы), основы (стема) и грамматических характеристик слова
* Синтез слова - генерация слова по заданным грамматическим характеристикам

### Морфологический процессор - инструмент морфологического анализа
* Морфологический словарь
* Морфологический анализатор

### Лемматизация
У каждого слова есть лемма (нормальная форма)
* кошке, кошку, кошкам, кошкой $\implies$ кошка
* бежал, бежит, бегу $\implies$ бежать
* белому, белым, белыми $\implies$ белый

In [ ]:
sent1 = 'Действительно, на его лице не отражалось никаких чувств – ни проблеска сочувствия не было на нем, \
а ведь боль просто невыносима'
sent2 = 'У страха глаза велики .'

In [ ]:
from pymorphy2 import MorphAnalyzer

m = MorphAnalyzer()
lemma1 = [m.parse(word)[0].normal_form for word in sent1.split()]
print(' '.join(lemma1))

In [ ]:
from pymystem3 import Mystem

m  = Mystem()
lemma2 = m.lemmatize(sent1)
print(''.join(lemma2))

### Стемминг
Слова состоят из морфем:  $word = stem + affixes$. Стемминг позволяет отбросить аффиксы. Чаще всего используется алгоритм Портера.
* 1-ый вид ошибки: белый, белка, белье $\implies$  бел

* 2-ой вид ошибки: трудность, трудный $\implies$  трудност, труд 

* 3-ий вид ошибки: быстрый, быстрее $\implies$  быст, побыстрее $\implies$  побыст

Алгоритм Портера состоит из 5 циклов команд, на каждом цикле – операция удаления / замены суффикса. Возможны вероятностные расширения алгоритма.

In [ ]:
from nltk.stem.snowball import RussianStemmer

stemmer = RussianStemmer()
words = ['распределение', 'приставить', 'сделала', 'словообразование']
for w in words:
    stem = stemmer.stem(w)
    print(stem)

#### Разбор слова 

In [ ]:
word1 = 'ГАИ'

In [ ]:
m = MorphAnalyzer()
m.parse(word1)

In [ ]:
m = Mystem()
m.analyze(word1)

#### Задание 4
Найти в списке персонажей "Война и мир" (task3.txt) все уникальные женские имена

### Первичная обработка текста

#### Удаление стоп-слов

In [ ]:
# m = Mystem()
m = MorphAnalyzer()
text = ' '.join([line.strip() for line in open('datasets/nlp/task3.txt', encoding='utf8')])
tokens = words_only(text)

#  [{'analysis': [{'lex': 'вера', 'wt': 0.6768655918, 'gr': 'S,имя,жен,од=им,ед'}], 'text': 'Вера'}, {'text': '\n'}]
names = set()
prog = re.compile('[А-Я]{1}[а-я]+') # Слово с заглавной буквы
tokens = prog.findall(text)
lemmas = [m.parse(word)[0].normal_form for word in tokens]

for word in lemmas:
    if {'Name','femn'} in m.parse(word)[0].tag:
        names.add(word.capitalize())
        
print(names)

In [ ]:
from nltk.corpus import stopwords

print(stopwords.words('Russian'))

In [ ]:
mystopwords = stopwords.words('russian') + ['это', 'наш' , 'тыс', 'млн', 'млрд', 'также',  'т', 'д']
def remove_stopwords(text, mystopwords=mystopwords):
    try:
        return ' '.join([token for token in text.split() if token in mystopwords])
    except:
        return ''

In [ ]:
m = Mystem()
def lemmatize(text, mystem=m):
    try:
        return ''.join(m.lemmatize(text)).strip()
    except:
        return ' '

In [ ]:
mystoplemmas = ['который','прошлый','сей', 'свой', 'наш', 'мочь']
def remove_stoplemmas(text, mystoplemmas=mystoplemmas):
    try:
        return " ".join([token for token in text.split() if not token in mystoplemmas])
    except:
        return ""


In [ ]:
from tqdm import tqdm
df.text = tqdm(df.text.apply(remove_stopwords))
df.text = tqdm(df.text.apply(lemmatize))
df.text = tqdm(df.text.apply(remove_stoplemmas))

In [ ]:
df.text

In [ ]:
lemmata = []
for index, row in df.iterrows():
    lemmata += row['text'].split()

fd = FreqDist(lemmata)
for i in fd.most_common(10):
    print(i)